In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os

In [4]:
os.chdir("./gdrive/")
os.chdir("./MyDrive/")
os.chdir("./Hack_uns")

In [30]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torchsummary as summary
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
import torch.nn.functional as F
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import time
import copy
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
data_dir = './Datasets'
classes = os.listdir(data_dir)
classes

['warts',
 'vitiligo',
 'acne-scars',
 'acne',
 'oily',
 'acanthosis-nigricans',
 'dry',
 'melasma',
 'alopecia-areata']

In [7]:
def get_transforms(mode='train'):
    transform = []
    transform.append(T.Resize((512, 512)))
    transform.append(T.ToTensor())
#     transform.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transform)

In [8]:
dataset = ImageFolder(data_dir, transform=get_transforms())
dataset

Dataset ImageFolder
    Number of datapoints: 908
    Root location: ./Datasets
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [9]:
len(dataset)

908

In [10]:
img, label = dataset[2]
print(img.shape, label)
img

torch.Size([3, 512, 512]) 0


tensor([[[0.0588, 0.0549, 0.0353,  ..., 0.0784, 0.0745, 0.0667],
         [0.0549, 0.0471, 0.0314,  ..., 0.0784, 0.0863, 0.0824],
         [0.0353, 0.0275, 0.0275,  ..., 0.0824, 0.0824, 0.0863],
         ...,
         [0.5647, 0.5686, 0.5725,  ..., 0.1020, 0.0941, 0.0980],
         [0.5647, 0.5765, 0.5725,  ..., 0.1137, 0.0902, 0.0902],
         [0.5725, 0.5765, 0.5725,  ..., 0.1020, 0.0980, 0.0902]],

        [[0.3608, 0.3569, 0.3569,  ..., 0.4510, 0.4471, 0.4431],
         [0.3569, 0.3569, 0.3529,  ..., 0.4510, 0.4471, 0.4431],
         [0.3569, 0.3529, 0.3490,  ..., 0.4510, 0.4510, 0.4510],
         ...,
         [0.5216, 0.5176, 0.5176,  ..., 0.4157, 0.4118, 0.4118],
         [0.5216, 0.5255, 0.5176,  ..., 0.4118, 0.4078, 0.4078],
         [0.5216, 0.5255, 0.5137,  ..., 0.4118, 0.4078, 0.4118]],

        [[0.3137, 0.3059, 0.3059,  ..., 0.4471, 0.4392, 0.4353],
         [0.3020, 0.3059, 0.3020,  ..., 0.4431, 0.4431, 0.4392],
         [0.3059, 0.3020, 0.2980,  ..., 0.4392, 0.4392, 0.

In [11]:
random_seed = 42
torch.manual_seed(random_seed);
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(727, 181)

In [12]:
efficientnet_b0 = models.efficientnet_b0(pretrained=True)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [13]:
efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [14]:
efficientnet_b0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=9, bias=True)
)

In [15]:
efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [16]:
model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

4019077

In [17]:
out = efficientnet_b0(torch.randn((1,3,512,512)))
len(out[0])

9

In [18]:
out

tensor([[-0.1611,  0.0530,  0.0935, -0.1472, -0.0036, -0.0634, -0.1190, -0.0301,
          0.1246]], grad_fn=<AddmmBackward0>)

In [19]:
for param in efficientnet_b0.features[:7].parameters():
    param.requires_grad = False

In [20]:
model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

1140921

In [21]:
params

1140921

In [22]:
from torch.utils.data.dataloader import DataLoader

batch_size=32

In [23]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

In [24]:
dataloaders = {'train':train_dl, 'val':val_dl}

In [25]:
dataset_sizes = {'train': len(train_ds), 'val': len(val_ds)}
dataset_sizes

{'train': 727, 'val': 181}

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [27]:
model = efficientnet_b0.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [28]:
# Fn for the training loop
def train_model(model, optimizer, alpha=1, beta=1, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print("-"*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            if phase == 'val':
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for imgs, labels in dataloaders[phase]:
                imgs = imgs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imgs)
                    _, preds = torch.max(outputs, 1)
                    loss = F.cross_entropy(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * imgs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            writer.add_scalar("Loss/train", epoch_loss, epoch)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [32]:
model = train_model(model, optimizer, num_epochs=25)

Epoch 1/25
----------
train Loss: 0.5957 Acc: 0.8528
val Loss: 0.5596 Acc: 0.8729

Epoch 2/25
----------
train Loss: 0.4098 Acc: 0.8913
val Loss: 0.4130 Acc: 0.8950

Epoch 3/25
----------
train Loss: 0.3077 Acc: 0.9326
val Loss: 0.3432 Acc: 0.9171

Epoch 4/25
----------
train Loss: 0.2400 Acc: 0.9422
val Loss: 0.2889 Acc: 0.9171

Epoch 5/25
----------
train Loss: 0.1815 Acc: 0.9560
val Loss: 0.2546 Acc: 0.9282

Epoch 6/25
----------
train Loss: 0.1438 Acc: 0.9670
val Loss: 0.2327 Acc: 0.9171

Epoch 7/25
----------
train Loss: 0.1390 Acc: 0.9615
val Loss: 0.2199 Acc: 0.9337

Epoch 8/25
----------
train Loss: 0.1087 Acc: 0.9780
val Loss: 0.2347 Acc: 0.9227

Epoch 9/25
----------
train Loss: 0.0968 Acc: 0.9766
val Loss: 0.1940 Acc: 0.9392

Epoch 10/25
----------
train Loss: 0.0851 Acc: 0.9807
val Loss: 0.2235 Acc: 0.9171

Epoch 11/25
----------
train Loss: 0.0727 Acc: 0.9807
val Loss: 0.1958 Acc: 0.9282

Epoch 12/25
----------
train Loss: 0.0662 Acc: 0.9821
val Loss: 0.1954 Acc: 0.9282

E

In [33]:
torch.save(model, "./skin-model.pt")